In [4]:
#!/usr/bin/env python
# coding: utf-8

import os
import torch, pyro, numpy as np
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from torch import tensor
import torch.nn as nn
import torchvision.transforms.functional as TF

import click


import swyft

DEVICE = 'cuda'

from utils import *

In [5]:
m = 1
nsub = 3
nsim = 10000

In [8]:


SYSTEM_NAME = "ngc4414"
RUN = f'_M_m{m}_nsub{nsub}_nsim{nsim}'
SIM_PATH = f'/nfs/scratch/eliasd/store{RUN}.zarr' 
print('run', RUN)


# Set utilities
torch.set_default_tensor_type(torch.cuda.FloatTensor)  # HACK
CONFIG = get_config(SYSTEM_NAME, str(nsub), str(m))
torch.set_default_tensor_type(torch.FloatTensor)

torch.set_default_tensor_type(torch.cuda.FloatTensor)  # HACK
ppd = CONFIG.ppd()['model_trace'].nodes
torch.set_default_tensor_type(torch.FloatTensor)


# Check simulation
v = ppd['main/sub/p_sub']['value']
sim = simul(v, CONFIG)

print('v', v)
print('sim', sim)

print("sim['image'] min and max" , sim['image'].min(), sim['image'].max() )


prior, uv = get_prior(CONFIG)
nx = CONFIG.kwargs["defs"]["nx"]
ny = CONFIG.kwargs["defs"]["ny"]

assert nsub == CONFIG.umodel.alphas["main"].sub.nsub
if m > 4:
    assert all([i == pow(10, m) for i in ppd['main/sub/m_sub']['value']])
else:
    print(f'm = {m} <= 0!', ppd['main/sub/m_sub']['value'])


# Create Store
pnames = [f'{z}_{i+1}' for i in range(nsub) for z in ['x', 'y', 'm']]
print(pnames)
simulator = swyft.Simulator(model = lambda v: simul(v, CONFIG), 
#                             pnames = ["x_sub", "y_sub"],
                            pnames = pnames,
                            sim_shapes={"image": (nx, ny)})

store = swyft.DirectoryStore(path=SIM_PATH, simulator=simulator)

store.add(nsim, prior)
store.simulate()

print('Done!')


run _M_m1_nsub3_nsim10000
v tensor([[ 1.7945,  0.1079],
        [ 1.1607,  2.2928],
        [-1.0019, -0.3892]], device='cuda:0')
sim {'image': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')}
sim['image'] min and max tensor(0., device='cuda:0') tensor(26.9432, device='cuda:0')
m = 1 <= 0! tensor([1.6384e+10, 5.7589e+11, 3.9214e+10], device='cuda:0')
['x_1', 'y_1', 'm_1', 'x_2', 'y_2', 'm_2', 'x_3', 'y_3', 'm_3']
Creating new store.
Store: Adding 10042 new samples to simulator store.
Done!
